In [51]:
# ZELLE A – Settings (alles zentral gesammelt)

import os
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta

load_dotenv()

# Zugangsdaten aus .env
API_KEY  = os.getenv("CAPITAL_API_KEY")
USERNAME = os.getenv("CAPITAL_USERNAME")
PWD      = os.getenv("CAPITAL_PASSWORD")
ACCOUNT  = os.getenv("CAPITAL_ACCOUNT_TYPE", "live")  # "demo" oder "live"

print("API-Key vorhanden?", bool(API_KEY))
print("Username:", USERNAME)
print("Account-Typ:", ACCOUNT)

# Basis-URLs
BASE_REST   = "https://api-capital.backend-capital.com"
BASE_STREAM = "wss://api-streaming-capital.backend-capital.com/connect"

# Instrumente
INSTRUMENTS = ["BTCUSD", "ETHUSD"]

# Hilfsfunktion Zeitformatierung
def to_local_time(ts: int) -> str:
    dt = datetime.fromtimestamp(ts/1000, tz=timezone.utc) + timedelta(hours=1)  # UTC+1
    return dt.strftime("%d.%m.%Y %H:%M:%S")


API-Key vorhanden? True
Username: carsten.schoettke@gmx.de
Account-Typ: live


In [52]:
# ZELLE B – Login und Session-Tokens holen

import requests

headers = {
    "X-CAP-API-KEY": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json"
}
payload = {
    "identifier": USERNAME,
    "password": PWD,
    "encryptedPassword": False
}

r = requests.post(f"{BASE_REST}/api/v1/session", headers=headers, json=payload)
print("Login HTTP:", r.status_code, r.reason)

CST  = r.headers.get("CST")
XSEC = r.headers.get("X-SECURITY-TOKEN")
print("CST vorhanden?", bool(CST), "XSEC vorhanden?", bool(XSEC))


Login HTTP: 200 
CST vorhanden? True XSEC vorhanden? True


In [53]:
# ZELLE C — Ticks -> eigene 1m-Kerzen pro Instrument (Europe/Berlin). Keine Dateien.

import json, math, asyncio, websockets
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# Candle-Historie für spätere Signal-Logik
candle_history = {epic: [] for epic in INSTRUMENTS}

# ---- Lokalzeit (DST-fähig) ----
LOCAL_TZ = ZoneInfo("Europe/Berlin")

def to_local_dt(ms_since_epoch: int) -> datetime:
    """Capital-Timestamp (ms, UTC) -> aware datetime in Europe/Berlin."""
    return datetime.fromtimestamp(ms_since_epoch/1000, tz=timezone.utc).astimezone(LOCAL_TZ)

def local_minute_floor(ts_ms: int) -> datetime:
    """Rundet auf die lokale Minuten-Grenze (Europe/Berlin)."""
    dt_local = to_local_dt(ts_ms)
    return dt_local.replace(second=0, microsecond=0)  # -> datetime (local tz)

# --- Kerzen-Optionen ---
PRICE_BASIS   = "mid"   # "mid" | "bid" | "ask"
BARS_TO_PRINT = 5       # wie viele finalisierte 1m-Bars je Instrument ausgeben
FORMING_PRINT_EVERY = 20  # jede n-te Tick-Aktualisierung zeigen

async def run_candle_aggregator_per_instrument():
    ws_url = f"{BASE_STREAM}?CST={CST}&X-SECURITY-TOKEN={XSEC}"
    subscribe = {
        "destination": "marketData.subscribe",
        "correlationId": "candles",
        "cst": CST,
        "securityToken": XSEC,
        "payload": {"epics": INSTRUMENTS},
    }

    # Zustand je Instrument
    # states[epic] = {"minute": datetime, "bar": {...}, "closed": int}
    states = {epic: {"minute": None, "bar": None, "closed": 0} for epic in INSTRUMENTS}

    def all_done():
        return all(st["closed"] >= BARS_TO_PRINT for st in states.values())

    print("Verbinde:", ws_url)
    async with websockets.connect(ws_url) as ws:
        await ws.send(json.dumps(subscribe))
        print("Subscribed:", INSTRUMENTS)

        while not all_done():
            raw = await ws.recv()
            try:
                msg = json.loads(raw)
            except Exception:
                continue
            if msg.get("destination") != "quote":
                continue

            p = msg["payload"]
            epic = p.get("epic")
            if epic not in states:
                # nur die abonnierten bearbeiten
                continue

            # Ticks lesen
            try:
                bid = float(p["bid"])
                ask = float(p["ofr"])
                ts_ms = int(p["timestamp"])  # ms (UTC)
            except Exception:
                continue

            # Preisbasis
            if PRICE_BASIS == "bid":
                px = bid
            elif PRICE_BASIS == "ask":
                px = ask
            else:
                px = (bid + ask) / 2.0  # mid

            # Minute pro Instrument
            minute_key = local_minute_floor(ts_ms)
            st = states[epic]

            # neue Minute für dieses Instrument? -> vorherige Bar finalisieren
            if st["minute"] is not None and minute_key > st["minute"] and st["bar"] is not None:
                st["closed"] += 1
                bar = st["bar"]
                print(
                    f"\n✅ [{epic}] Closed 1m  {st['minute'].strftime('%d.%m.%Y %H:%M:%S %Z')}  "
                    f"O:{bar['open']:.2f} H:{bar['high']:.2f} L:{bar['low']:.2f} C:{bar['close']:.2f}  "
                    f"Ticks:{bar['ticks']}"
                )

                candle_history[epic].append({
                    "Open": bar["open"],
                    "High": bar["high"],
                    "Low": bar["low"],
                    "Close": bar["close"]
                })

                try:
                    on_candle_close(epic, bar)
                except NameError:
                    pass

                # neue Bar beginnen
                st["minute"] = minute_key
                st["bar"] = {"open": px, "high": px, "low": px, "close": px, "ticks": 1}

            else:
                # gleiche Minute (forming) oder erste Bar
                if st["minute"] is None:
                    st["minute"] = minute_key
                    st["bar"] = {"open": px, "high": px, "low": px, "close": px, "ticks": 1}
                else:
                    b = st["bar"]
                    b["high"]  = max(b["high"], px)
                    b["low"]   = min(b["low"],  px)
                    b["close"] = px
                    b["ticks"] += 1

                try:
                    on_candle_forming(epic, st["bar"])
                except NameError:
                    pass

                # sparsamer forming-Print je Instrument
                if st["bar"]["ticks"] % FORMING_PRINT_EVERY == 1:
                    print(
                        f"… forming [{epic}] {to_local_dt(ts_ms).strftime('%d.%m.%Y %H:%M:%S %Z')}  "
                        f"O:{st['bar']['open']:.2f} H:{st['bar']['high']:.2f} "
                        f"L:{st['bar']['low']:.2f} C:{st['bar']['close']:.2f}  "
                        f"Ticks:{st['bar']['ticks']}"
                    )

import asyncio
asyncio.run(run_candle_aggregator_per_instrument())



Verbinde: wss://api-streaming-capital.backend-capital.com/connect?CST=mhKezkejzla4jGSjy0u1NsQR&X-SECURITY-TOKEN=kArfyIq3RohWcxHGZgWBuHjQ3BpBLLW
Subscribed: ['BTCUSD', 'ETHUSD']
… forming [BTCUSD] 21.09.2025 00:30:39 CEST  O:115835.35 H:115835.35 L:115835.35 C:115835.35  Ticks:1
… forming [ETHUSD] 21.09.2025 00:30:41 CEST  O:4496.06 H:4496.06 L:4496.06 C:4496.06  Ticks:1


KeyboardInterrupt: 

In [ ]:
# ZELLE D — Demo-Signal-Logik (ohne pandas_ta)

print("✅ Signal-Logik (Zelle D) geladen")

def on_candle_forming(epic, bar):
    """Wird bei jedem Tick innerhalb einer Kerze aufgerufen (noch nicht geschlossen)."""
    if bar["ticks"] % 50 == 0:
        print(f"🔄 Forming-Signal [{epic}] — C:{bar['close']:.2f} (Ticks:{bar['ticks']})")

def on_candle_close(epic, bar):
    """Wird aufgerufen, sobald eine 1m-Kerze abgeschlossen ist."""
    if bar["close"] > bar["open"]:
        signal = "BUY ✅"
    elif bar["close"] < bar["open"]:
        signal = "SELL ⛔"
    else:
        signal = "NEUTRAL ⚪"

    print(
        f"📊 Signal [{epic}] — O:{bar['open']:.2f} C:{bar['close']:.2f} → {signal}"
    )


✅ Signal-Logik (Zelle D) geladen
